In [ ]:
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup  
from urllib.request import urlopen

import re

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import datetime
import time

import geocoder
import wikipedia as wk

In [2]:
#!pip install geocoder

  Running setup.py bdist_wheel for future: started
  Running setup.py bdist_wheel for future: finished with status 'done'
  Stored in directory: C:\Users\LG_\AppData\Local\pip\Cache\wheels\c2\50\7c\0d83b4baac4f63ff7a765bd16390d2ab43c93587fac9d6017a
Successfully built future


In [ ]:
#!pip install wikipedia




# GoogleMap

In [8]:
#셀리옴으로 크롤링하기

options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
#options.add_argument("disable-gpu")
options.add_argument("--disable-gpu")

In [9]:
driver = webdriver.Chrome('C:\\ProgramData\\Anaconda3\\Lib\\site-packages\\selenium\\webdriver\\chrome\\chromedriver.exe', chrome_options=options)
time.sleep(3)

In [16]:
#나라별 도시 크롤링
country =  ["스페인", "포르투갈"]
dic = {}
for idx in country :
    dic[idx] = []

In [17]:
tot = 0
for nature in country :
    base_url = 'https://www.google.co.kr/search?' +'q=' + nature + '+도시&hl=en'
    driver.get(base_url)
    time.sleep(3)
    
    lst = driver.find_elements_by_css_selector("a.klitem")
    for city in lst :
        dic[nature].append(city.text.split("\n")[0])
        tot += 1
    
print(tot) 

In [18]:
regex = ">(.*)</span>"
result = {}
no_coordi_lst = []
err_lst = []
check = 0

for nature in country :
    for place in dic[nature] :
        tot = 0
        check += 1
        print(str(check) + " " + nature + " " + place, end = " ")
        
        try : 
            base_url = 'https://www.google.co.kr/maps/search/' + place + '+관광지/data=!4m3!2m2!5m1!4e1'
            driver.get(base_url)
            time.sleep(5)

            html =  driver.page_source
            soup = BeautifulSoup(html, "html.parser")

            latitude = 0
            longtitude = 0 
            url = driver.current_url

            if "/@" in url :
                url = re.search("/@(.*)z/", url).group(1).split(",")
                latitude = float(url[0])
                longtitude = float(url[1])
            else :
                no_coordi_lst.append((place, nature))

            place_lst = soup.select("body > jsl > div#app-container > div#content-container > div#pane > div.widget-pane.widget-pane-visible > div.widget-pane-content.scrollable-y > div.widget-pane-content-holder > div.section-listbox.section-listbox-root > div > div.section-result > div > div.section-result-header > div > h3")
            review_lst = soup.select("body > jsl > div#app-container > div#content-container > div#pane > div.widget-pane.widget-pane-visible > div.widget-pane-content.scrollable-y > div.widget-pane-content-holder > div.section-listbox.section-listbox-root > div > div.section-result > div > div.section-result-header > div > span > span.section-result-num-ratings")
            star_lst = soup.select("body > jsl > div#app-container > div#content-container > div#pane > div.widget-pane.widget-pane-visible > div.widget-pane-content.scrollable-y > div.widget-pane-content-holder > div.section-listbox.section-listbox-root > div > div.section-result > div > div.section-result-header > div > span > span > span")

            count = 0
            for idx1, idx2, idx3 in zip(place_lst, review_lst, star_lst) :
                result1 = re.search(regex, str(idx1)).group(1).split(">")[1]
                result2 = re.search(regex, str(idx2)).group(1).replace("(", "").replace(")" ,"").replace(",", "")
                if not result2 :
                    result2 = 0
                else :
                    result2 = int(result2)
                result3 = float(re.search(regex, str(idx3)).group(1))

                result[result1] = (place ,nature, result2, result3, latitude, longtitude)
                count += 1
                tot += 1

            flag = True
            if count != 20 :
                flag = False

            while flag :
                try :
                    next_button_id = 'section-pagination-button-next'
                    inputElement = driver.find_element_by_id(next_button_id)
                    inputElement.send_keys("\n")
                    time.sleep(5)

                    html = driver.page_source
                    soup = BeautifulSoup(html, "html.parser")

                    place_lst = soup.select("body > jsl > div#app-container > div#content-container > div#pane > div.widget-pane.widget-pane-visible > div.widget-pane-content.scrollable-y > div.widget-pane-content-holder > div.section-listbox.section-listbox-root > div > div.section-result > div > div.section-result-header > div > h3")
                    review_lst = soup.select("body > jsl > div#app-container > div#content-container > div#pane > div.widget-pane.widget-pane-visible > div.widget-pane-content.scrollable-y > div.widget-pane-content-holder > div.section-listbox.section-listbox-root > div > div.section-result > div > div.section-result-header > div > span > span.section-result-num-ratings")
                    star_lst = soup.select("body > jsl > div#app-container > div#content-container > div#pane > div.widget-pane.widget-pane-visible > div.widget-pane-content.scrollable-y > div.widget-pane-content-holder > div.section-listbox.section-listbox-root > div > div.section-result > div > div.section-result-header > div > span > span > span")

                    count = 0
                    for idx1, idx2, idx3 in zip(place_lst, review_lst, star_lst) :
                        result1 = re.search(regex, str(idx1)).group(1).split(">")[1]
                        result2 = re.search(regex, str(idx2)).group(1).replace("(", "").replace(")" ,"").replace(",", "")
                        if not result2 :
                            result2 = 0
                        else :
                            result2 = int(result2)
                        result3 = float(re.search(regex, str(idx3)).group(1))

                        result[result1] = (place ,nature, result2, result3, latitude, longtitude)
                        count += 1
                        tot += 1

                    if count != 20 :
                        flag = False

                except :
                    print("err1", end=" ")
                    flag = False
                    
                    
            print(tot, "clear")
            
        except :
            err_lst.append((place, nature))
            print(tot, "err2")

                
            
            

1 스페인 Barcelona 128 clear
2 스페인 Madrid 114 clear
3 스페인 Seville 84 clear
4 스페인 Granada 61 clear
5 스페인 Valencia 85 clear
6 스페인 Málaga 1 clear
7 스페인 Bilbao 20 clear
8 스페인 Toledo 36 clear
9 스페인 Córdoba 50 clear
10 스페인 Segovia 10 clear
11 스페인 Palma, Majorca 43 clear
12 스페인 Zaragoza 29 clear
13 스페인 Santiago de Compostela 17 clear
14 스페인 Ronda 15 clear
15 스페인 Alicante 73 clear
16 스페인 Ceuta 7 clear
17 스페인 Melilla 3 clear
18 스페인 Santander 9 clear
19 스페인 Donostia / San Sebastián err1 120 clear
20 스페인 Salamanca 17 clear
21 스페인 Las Palmas 37 clear
22 스페인 Gijón 8 clear
23 스페인 León 11 clear
24 스페인 Cádiz 41 clear
25 스페인 Burgos 5 clear
26 스페인 Vigo 7 clear
27 스페인 Oviedo 9 clear
28 스페인 Elche 3 clear
29 스페인 Valladolid 14 clear
30 스페인 Pamplona 9 clear
31 스페인 Irun 2 clear
32 스페인 Mérida 2 clear
33 스페인 A Coruña 10 clear
34 스페인 Cuenca 13 clear
35 스페인 Ferrol 7 clear
36 스페인 Almería 19 clear
37 스페인 Guadalajara 33 clear
38 스페인 Ávila, Spain 1 clear
39 스페인 Santa Cruz de Tenerife 138 clear
40 스페인 Castellón de la Pla

In [19]:
driver.quit()

In [20]:
df_city_info = pd.DataFrame.from_items(list(result.items())).T

In [21]:
print(len(df_city_info))
df_city_info.head(10)

1494


,0,1,2,3,4,5
City Sightseeing Barcelona,Barcelona,스페인,5,3.6,41.392,2.18339
Barcelona City Tour,Barcelona,스페인,113,3.2,41.392,2.18339
Barcelona Bus Turístic,Barcelona,스페인,78,2.8,41.392,2.18339
Running Tours Barcelona - Sightseeing and Running in Barcelona,Barcelona,스페인,4,5,41.392,2.18339
레이알광장,Barcelona,스페인,2598,4.4,41.392,2.18339
GoCar Barcelona,Barcelona,스페인,39,3.5,41.392,2.18339
Barcelona Day Tours,Barcelona,스페인,322,5,41.392,2.18339
Discover Walks Barcelona - Barcelona Walking Tours,Barcelona,스페인,8,5,41.392,2.18339
Catamaran Orsom,Barcelona,스페인,20,4.4,41.392,2.18339
Runner Bean Tours,Barcelona,스페인,44,4.9,41.392,2.18339


In [22]:
#df_city_info.to_csv("travel_info3.csv")





# 구글 관광명소
### (스페인 도시 검색 -> 스페인 ~~~ 여행 -> 관광명소 크롤링, 만약 관광명소 없으면 우측에서 크롤링 -> 없으면 예외)


In [2]:
#셀리옴으로 크롤링하기

options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
#options.add_argument("disable-gpu")
options.add_argument("--disable-gpu")

In [13]:
driver = webdriver.Chrome('C:\\ProgramData\\Anaconda3\\Lib\\site-packages\\selenium\\webdriver\\chrome\\chromedriver.exe', chrome_options=options)
time.sleep(3)

In [4]:
#나라별 도시 크롤링
country =  ["스페인", "포르투갈"]
dic = {}
for idx in country :
    dic[idx] = []

In [5]:
tot = 0
for nature in country :
    base_url = 'https://www.google.co.kr/search?' +'q=' + nature + '+도시'
    driver.get(base_url)
    time.sleep(3)
    
    lst = driver.find_elements_by_css_selector("a.klitem")
    for city in lst :
        dic[nature].append(city.text.split("\n")[0])
        tot += 1
    
print(tot)

102


In [14]:
result = []
err_lst = []

regex1 = ">(.*)<"
regex2 = "중 (.*)개"
regex3 = "리뷰 (.*)개"
regex4 = ">(.*)<"
regex5 = ""

tot = 1
a = 0
total = 0
for nature in country :
    for city in dic[nature] :
        print(tot, nature, city, end = " ")
        a+=1
        
        try : 
            base_url = 'https://www.google.co.kr/search?' +'q=' + nature + '+' + city + '+' + "관광지"
            driver.get(base_url)
            time.sleep(5)
            
            more_button_id = '_d9n'
            driver.find_element_by_id(more_button_id).click()
            time.sleep(5)

            html =  driver.page_source
            soup = BeautifulSoup(html, "html.parser")
            temp = soup.select("body > div > div > div > ol > li > div > div")
            
            total = 0
            for place in temp :
                soup = BeautifulSoup(str(place), "html.parser")
                
                name = re.search(regex1, str(soup.select("div > h2")[0])).group(1)
                
                try : star = float(re.search(regex2, str(soup.select("div >  g-review-stars > span")[0])).group(1))
                except : star = 0
                    
                try : review = int(re.search(regex3, str(soup.select("div > div  > span._Mnc")[0])).group(1).replace(",", ""))
                except : review = 0
                
                try : short_info = re.search(regex4, str(soup.select("p._DAm")[0])).group(1)
                except : short_info = ""
                    
                try : long_info = str(soup.select("p._BAm")[0]).split(">")[1].split("<")[0]
                except : long_info = ""
                    
                result.append((name, (nature, city, star, review, short_info, long_info)))
                total += 1
                
            print(total, "clear")
        except :
            err_lst.append((nature, city))
            print("err")
            
        tot+=1

        


        
        

1 스페인 바르셀로나 50 clear
2 스페인 마드리드 50 clear
3 스페인 세비야 50 clear
4 스페인 그라나다 50 clear
5 스페인 발렌시아 50 clear
6 스페인 말라가 50 clear
7 스페인 빌바오 50 clear
8 스페인 톨레도 50 clear
9 스페인 코르도바 50 clear
10 스페인 세고비아 50 clear
11 스페인 사라고사 50 clear
12 스페인 팔마데마요르카 50 clear
13 스페인 론다 50 clear
14 스페인 산티아고데콤포스텔라 50 clear
15 스페인 알리칸테 50 clear
16 스페인 세우타 50 clear
17 스페인 멜리야 35 clear
18 스페인 산세바스티안 50 clear
19 스페인 산탄데르 50 clear
20 스페인 살라망카 50 clear
21 스페인 라스팔마스 50 clear
22 스페인 히혼 50 clear
23 스페인 레온 50 clear
24 스페인 카디스 50 clear
25 스페인 부르고스 50 clear
26 스페인 비고 50 clear
27 스페인 오비에도 50 clear
28 스페인 엘체 44 clear
29 스페인 바야돌리드 50 clear
30 스페인 팜플로나 50 clear
31 스페인 이룬 50 clear
32 스페인 메리다 50 clear
33 스페인 라코루냐 50 clear
34 스페인 쿠엥카 44 clear
35 스페인 과달라하라 38 clear
36 스페인 페롤 50 clear
37 스페인 알메리아 50 clear
38 스페인 아빌라 45 clear
39 스페인 산타크루스데테네리페 50 clear
40 스페인 카스테욘데라플라나 50 clear
41 스페인 알헤시라스 36 clear
42 스페인 비토리아 50 clear
43 스페인 루고 39 clear
44 스페인 카세레스 50 clear
45 스페인 게르니카 8 clear
46 스페인 사군토 42 clear
47 스페인 아빌레스 48 clear
48 스페인 푸에르토레알 50 clear


In [46]:
driver.quit()

In [15]:
df_city_google_info = pd.DataFrame.from_items(result).T

In [16]:
print(len(df_city_google_info))
df_city_google_info.tail(10)

3954


,0,1,2,3,4,5
Praia Velha,포르투갈,이스토릴,4.1,92,,
Chafariz de Paço de Arcos,포르투갈,이스토릴,0,0,,
Praia dos Estoris,포르투갈,이스토릴,0,0,,
Fornos da Cal,포르투갈,이스토릴,0,0,,
Praia da Ribeira de Cascais,포르투갈,이스토릴,0,0,,
São Vicente Caves,포르투갈,상비센트,4.1,526,동굴 및 화산,
Casas Típicas de Santana,포르투갈,상비센트,4.2,92,,
HP16 Levada Faja do Rodrigues,포르투갈,상비센트,4.7,16,,
Rota da Cal,포르투갈,상비센트,4.9,9,,
Igreja Matriz de São Vicente,포르투갈,상비센트,4.4,15,교회당,


In [18]:
#직접 url 들어가서 노가다
#벨라스는 검색결과 없음
err_lst

[('포르투갈', '로르스'),
 ('포르투갈', '산타렝'),
 ('포르투갈', '벨라스'),
 ('포르투갈', '아비스'),
 ('포르투갈', '모라'),
 ('포르투갈', '이스포젠드')]

In [ ]:
#df_city_google_info.to_csv('travel_google_info4.csv')

# 좌표 받기
### (노가다 ver)

In [2]:
#셀리옴으로 크롤링하기

options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
#options.add_argument("disable-gpu")
options.add_argument("--disable-gpu")

In [50]:
driver = webdriver.Chrome('C:\\ProgramData\\Anaconda3\\Lib\\site-packages\\selenium\\webdriver\\chrome\\chromedriver.exe', chrome_options=options)
time.sleep(3)

In [ ]:
coordi_lst = []
no_coordi_lst = []
cnt = 0

for desti in result :
    target_place = desti[0]
    target_city = desti[1][1]
    
    if cnt%20 == 0 :
        print("-----------------",cnt,"-----------------")
    cnt+=1
    
    try :
        base_url = 'https://www.google.co.kr/maps/search/' + target_city + '+' + target_place
        driver.get(base_url)
        time.sleep(5)

        latitude = 0
        longtitude = 0 
        url = driver.current_url

        if "/@" in url :
            try :
                url = re.search("/@(.*)z/", url).group(1).split(",")
                latitude = float(url[0])
                longtitude = float(url[1])
            except :
                latitude = url
                longtitude = url
                no_coordi_lst.append((target_place, target_city, url))
                print((target_place, targey_city))
            
        else :
            no_coordi_lst.append((target_place, target_city))
            print((target_place, targey_city))
        coordi_lst.append((target_place, (latitude, longtitude))) 
            
    except :
        print((target_place, target_city))
        



----------------- 0 -----------------
----------------- 20 -----------------
('바르셀로나 현대미술관', '바르셀로나')
----------------- 40 -----------------
----------------- 60 -----------------
----------------- 80 -----------------
----------------- 100 -----------------
----------------- 120 -----------------
----------------- 140 -----------------
----------------- 160 -----------------
----------------- 180 -----------------
('Santa Isabel La Real', '그라나다')
----------------- 200 -----------------
----------------- 220 -----------------
('Palacio Baylía', '발렌시아')
----------------- 240 -----------------
----------------- 260 -----------------
----------------- 280 -----------------
('Guadalmedina', '말라가')
----------------- 300 -----------------
----------------- 320 -----------------
----------------- 340 -----------------
----------------- 360 -----------------
----------------- 380 -----------------
----------------- 400 -----------------
----------------- 420 -----------------
-----------------

In [49]:
driver.quit()

In [ ]:
df_place_coodi_info = pd.DataFrame.from_items(coordi_lst).T

In [ ]:
print(len(df_place_coodi_info))
df_place_coodi_info.tail(10)

In [ ]:
no_coordi_lst

In [ ]:
df_place_coodi_info.to_csv('travel_google_info4.csv')

# 지도받기
### (geocoder 이용)

In [2]:
result = pd.read_csv('travel_google_info3.csv', encoding='cp949')

In [3]:
result.head(3)

,Unnamed: 0,0,1,2,3,4,5
0,사그라다 파밀리아,스페인,바르셀로나,4.7,11013,"안토니 가우디, 바실리카, 교회당 및 건축",사그라다 파밀리아 성당은 스페인 바르셀로나에 짓고 있는 로마 가톨릭 성당이다.\n카...
1,구엘 공원,스페인,바르셀로나,4.3,21144,공원,구엘 공원은 스페인 바르셀로나에 있는 공원이다. 1984년 유네스코 세계유산에 등록...
2,카사 밀라,스페인,바르셀로나,4.4,8547,안토니 가우디,카사밀라는 안토니오 가우디의 작품으로 바르셀로나 중심가인 그라시아 거리에 있다. '...


In [4]:
result = result.values.tolist()

In [ ]:
coordi_lst = []
no_coordi_lst = []
cnt = 0

for desti in result :
    target_place = desti[0]
    target_city = desti[2]
    target_nature = desti[1]
    
    if cnt%20 == 0 :
        print("-----------------",cnt,"-----------------")
    cnt+=1
    
    latitude = 0
    longtitude = 0 
    
    try :
        g = geocoder.google(target_city + " " + target_place)
        time.sleep(2)
        
        trial = 0
        while (g.latlng == None) :
            g = geocoder.google(target_city + " " + target_place)
            time.sleep(2)
            trial += 1
            
            if trial >= 3 :
                break
                
        latitude, longtitude = g.latlng
        
        coordi_lst.append((target_place, (target_nature, target_city, latitude, longtitude))) 
            
    except :
        no_coordi_lst.append((target_place, target_city, target_nature))
        coordi_lst.append((target_place, (target_nature, target_city, latitude, longtitude))) 
        print(cnt, (target_place, target_city, target_nature))
    
    if cnt%400 == 0 :
        df_place_coodi_info = pd.DataFrame.from_items(coordi_lst).T
        df_place_coodi_info.to_csv('travel_google_coordinate' + str(cnt) + '.csv')
        print("saved")
    
df_place_coodi_info = pd.DataFrame.from_items(coordi_lst).T
df_place_coodi_info.to_csv('travel_google_coordinate' + str(cnt) + '.csv')
print("----------------- finish -----------------")

----------------- 0 -----------------


Status code Unknown from https://maps.googleapis.com/maps/api/geocode/json: ERROR - HTTPSConnectionPool(host='maps.googleapis.com', port=443): Read timed out. (read timeout=5.0)


3 ('카사 밀라', '바르셀로나')


Status code Unknown from https://maps.googleapis.com/maps/api/geocode/json: ERROR - HTTPSConnectionPool(host='maps.googleapis.com', port=443): Read timed out. (read timeout=5.0)


4 ('람블라 거리', '바르셀로나')


...

12 ('바르셀로네타', '바르셀로나')


Status code Unknown from https://maps.googleapis.com/maps/api/geocode/json: ERROR - HTTPSConnectionPool(host='maps.googleapis.com', port=443): Read timed out. (read timeout=5.0)


14 ('스페인 촌', '바르셀로나')


Status code Unknown from https://maps.googleapis.com/maps/api/geocode/json: ERROR - HTTPSConnectionPool(host='maps.googleapis.com', port=443): Read timed out. (read timeout=5.0)


15 ('카탈라냐 음악당', '바르셀로나')


In [ ]:
df_place_coodi_info = pd.DataFrame.from_items(coordi_lst).T

In [ ]:
print(len(df_place_coodi_info))
df_place_coodi_info.tail(10)

In [ ]:
no_coordi_lst

In [ ]:
df_place_coodi_info.to_csv('travel_google_coordinate.csv')

# 리뷰받기
### (구글지도url에서 검색 ->리뷰보기->영어? 한국어? )

In [2]:
#셀리옴으로 크롤링하기

options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
#options.add_argument("disable-gpu")
options.add_argument("--disable-gpu")

In [3]:
driver = webdriver.Chrome('C:\\ProgramData\\Anaconda3\\Lib\\site-packages\\selenium\\webdriver\\chrome\\chromedriver.exe', chrome_options=options)
time.sleep(3)

In [4]:
df = pd.read_csv('travel_google_info3.csv', encoding='cp949')
df_lst = df.values.tolist()[60:]

In [5]:
result = []
err = []
cnt = 60
print("start")

for idx in df_lst :
    place = idx[0]
    nature = idx[1]
    city = idx[2]
    review_num = idx[4]
    
    if cnt%10 == 0 :
        print("-----------------",cnt,"-----------------")
    cnt += 1
    
    #영어 리뷰를 받을거면 ?hl=en을 붙여줘야함
    base_url = 'https://www.google.co.kr/maps/search/' + city + " " + place
    driver.get(base_url)
    time.sleep(5)
    
    #좌표받기
    latitude = 0
    longtitude = 0 
    
    url = driver.current_url
    if "/@" in url :
        try :
            url = re.search("/@(.*)z/", url).group(1).split(",")
            latitude = float(url[0])
            longtitude = float(url[1])
            
        except :
            latitude = url
            longtitude = url
            err.append((nature, place, city, url))
            print(cnt, "url error", (nature, place, city))
            
    else :
        err.append((nature, place, city, url))
        print(cnt, "no coordinate", (nature, place, city))

    #리뷰받기
    if review_num == 0 :
        result.append([place, nature, city, (latitude, longtitude),[""]])
        continue
        
    time.sleep(3)
    
    flag = False
    try :
        driver.find_element_by_css_selector('button.widget-pane-link').click()
        time.sleep(3)

    except :
        try :
            driver.find_element_by_xpath("//div[@data-result-index='1'][@class='section-result']").click()
            print(cnt, "no found", (nature, place, city))
            time.sleep(3)
            flag = True
            
        except :
            print(cnt, "no review button", (nature, place, city))
            result.append([place, nature, city, (latitude, longtitude),[""]])
            err.append((nature, place, city, url))
            continue
        
    if flag : 
        try :
            driver.find_element_by_css_selector('button.widget-pane-link').click()
            time.sleep(3)
            
        except :
            print(cnt, "no review button", (nature, place, city))
            result.append([place, nature, city, (latitude, longtitude),[""]])
            err.append((nature, place, city, url))
            continue
        
    
    

    for idx in range(10) :
        try :
            driver.find_element_by_css_selector('div.section-loading.noprint').click()
            time.sleep(3)
        except :
            break
    
    html =  driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    review_lst = soup.select('body > jsl > div > div > div > div > div > div > div > div > div > div > div > div > div > div > span')
    
    if len(review_lst) == 0 :
        result.append([place, nature, city, (latitude, longtitude),[""]])
    else :
        temp = []
        for review in review_lst :
            temp.append(str(review).split('>')[1].split('<')[0])
        
        result.append([place, nature, city, (latitude, longtitude), temp])
        
    if cnt%100 == 0 :
        
        new_result = list(map(lambda x : (x[0], (x[1], x[2], x[3], str(x[4]))), result))
        
        df_place_review = pd.DataFrame.from_items(new_result).T
        df_place_review.to_csv('travel_google_review' + str(cnt) + '.csv')
        print("saved")

        

print("finish")

start
----------------- 60 -----------------
68 no review button ('스페인', 'Madrid R?o', '마드리드')
----------------- 70 -----------------
----------------- 80 -----------------
86 no found ('스페인', 'Museum of L?zaro Galdiano', '마드리드')
----------------- 90 -----------------
98 no review button ('스페인', 'C?rculo de Bellas Artes', '마드리드')
saved
----------------- 100 -----------------
102 no found ('스페인', '세비야 대성당', '세비야')
106 no found ('스페인', 'Maestranza', '세비야')
----------------- 110 -----------------
111 no found ('스페인', 'Isla M?gica', '세비야')
111 no review button ('스페인', 'Isla M?gica', '세비야')
117 no found ('스페인', 'La Alameda, Seville', '세비야')
119 no found ('스페인', 'Palacio de las Due?as', '세비야')
----------------- 120 -----------------
129 no found ('스페인', 'Chapel of San Jos?', '세비야')
130 no found ('스페인', 'Centro de la Cer?mica de Triana', '세비야')
----------------- 130 -----------------
137 no review button ('스페인', 'Bas?lica de la Macarena', '세비야')
140 no found ('스페인', 'Plaza del Triunfo', '세비야'

334 no found ('스페인', 'Parroquia de "San Vicente M?rtir de Abando"', '빌바오')
335 no review button ('스페인', 'La gr?a Carola', '빌바오')
340 no found ('스페인', 'Museo de Reproducciones Art?sticas', '빌바오')
----------------- 340 -----------------
348 no found ('스페인', 'Mirador de Artxanda', '빌바오')
----------------- 350 -----------------
351 no found ('스페인', '톨레도 대성당', '톨레도')
356 no coordinate ('스페인', 'Santa Mar?a la Blanca', '톨레도')
356 no review button ('스페인', 'Santa Mar?a la Blanca', '톨레도')
360 no found ('스페인', 'Puerta de Bisagra', '톨레도')
----------------- 360 -----------------
370 no found ('스페인', 'Museo de "La Espa?a M?gica"', '톨레도')
----------------- 370 -----------------
376 no found ('스페인', 'Museo de "Tapices y Textiles de la Catedral de Toledo"', '톨레도')
379 no found ('스페인', 'Puente de Alc?ntara', '톨레도')
----------------- 380 -----------------
381 no found ('스페인', 'Zocodover', '톨레도')
386 url error ('스페인', 'Senda Ecol?gica De Toledo', '톨레도')
386 no review button ('스페인', 'Senda Ecol?gica De Tol

678 no review button ('스페인', 'Museo Centro Gai?s', '산티아고데콤포스텔라')
----------------- 680 -----------------
684 url error ('스페인', 'Convento de Belv?s', '산티아고데콤포스텔라')
684 no review button ('스페인', 'Convento de Belv?s', '산티아고데콤포스텔라')
685 no found ('스페인', 'Casa do Cabido. Sala de Exposici?ns', '산티아고데콤포스텔라')
687 no review button ('스페인', 'Museo e Iglesia de San Mart?n Pinario', '산티아고데콤포스텔라')
----------------- 690 -----------------
692 no found ('스페인', 'Capela de ?nimas', '산티아고데콤포스텔라')
693 no found ('스페인', 'CIPX Compostela (Centro de Interpretaci?n dos Parques e Xard?ns de Compostela)', '산티아고데콤포스텔라')
693 no review button ('스페인', 'CIPX Compostela (Centro de Interpretaci?n dos Parques e Xard?ns de Compostela)', '산티아고데콤포스텔라')
----------------- 700 -----------------
707 no found ('스페인', 'Concatedral de San Nicol?s, Alicante', '알리칸테')
----------------- 710 -----------------
716 no review button ('스페인', 'Playa Muchavista', '알리칸테')
717 no review button ('스페인', 'El Palmeral', '알리칸테')
----------------- 7

943 no review button ('스페인', 'Torre del Clavero', '살라망카')
944 no coordinate ('스페인', 'Museum of Automotive History', '살라망카')
944 no review button ('스페인', 'Museum of Automotive History', '살라망카')
945 no coordinate ('스페인', 'Convento de las ?rsulas', '살라망카')
945 no review button ('스페인', 'Convento de las ?rsulas', '살라망카')
946 no coordinate ('스페인', 'Museo de Salamanca', '살라망카')
946 no review button ('스페인', 'Museo de Salamanca', '살라망카')
947 no coordinate ('스페인', 'Museo Taurino de Salamanca', '살라망카')
947 no review button ('스페인', 'Museo Taurino de Salamanca', '살라망카')
948 no coordinate ('스페인', 'Palacio de Orellana', '살라망카')
948 no review button ('스페인', 'Palacio de Orellana', '살라망카')
949 no coordinate ('스페인', 'Museum of Commerce', '살라망카')
949 no review button ('스페인', 'Museum of Commerce', '살라망카')
950 no coordinate ('스페인', 'Domus Artium 2002', '살라망카')
950 no review button ('스페인', 'Domus Artium 2002', '살라망카')
----------------- 950 -----------------
951 no coordinate ('스페인', 'Tower Clerecia', '살라망카')

1175 no found ('스페인', 'Roman Catholic Diocese of C?diz y Ceuta', '카디스')
1177 no found ('스페인', 'Paseo Canalejas', '카디스')
----------------- 1180 -----------------
1181 url error ('스페인', 'Casa Palacio Ar?mburu', '카디스')
1181 no review button ('스페인', 'Casa Palacio Ar?mburu', '카디스')
1182 url error ('스페인', 'Playa de la Caser?a', '카디스')
1183 no found ('스페인', 'Jardines de Varela', '카디스')
1189 no found ('스페인', 'Museum of Human Evolution', '부르고스')
----------------- 1190 -----------------
1194 no found ('스페인', 'Monasterio de San Juan', '부르고스')
1198 url error ('스페인', 'Casa del Cord?n', '부르고스')
1198 no review button ('스페인', 'Casa del Cord?n', '부르고스')
saved
----------------- 1200 -----------------
1202 no found ('스페인', 'Mirador Del Castillo', '부르고스')
1206 url error ('스페인', 'Glera del Arlanz?n', '부르고스')
1208 no found ('스페인', 'Fuente de Santa Mar?a', '부르고스')
1210 no found ('스페인', 'Iglesia Santa Agueda', '부르고스')
----------------- 1210 -----------------
1213 url error ('스페인', 'Plaza de los Casta?os', '부르

...

3276 no review button ('포르투갈', 'Apdl - Administra??o dos Portos do Douro, Leix?es e Viana do Castelo, SA', '마토지뉴스')
3279 url error ('포르투갈', 'Paradise Beach', '마토지뉴스')
3279 no review button ('포르투갈', 'Paradise Beach', '마토지뉴스')
3280 url error ('포르투갈', 'Busto do Padre Diamantino Gomes', '마토지뉴스')
----------------- 3280 -----------------
3281 url error ('포르투갈', 'Parque Bas?lio Teles', '마토지뉴스')
3285 url error ('포르투갈', 'Pedra Romana', '마토지뉴스')
3285 no review button ('포르투갈', 'Pedra Romana', '마토지뉴스')
3286 url error ('포르투갈', 'Beach Funt?o', '마토지뉴스')
3286 no review button ('포르투갈', 'Beach Funt?o', '마토지뉴스')
3288 url error ('포르투갈', 'Fonte Luminosa', '마토지뉴스')
3288 no review button ('포르투갈', 'Fonte Luminosa', '마토지뉴스')
3290 url error ('포르투갈', 'Jardim Ant?nio Borges', '폰타델가다')
3290 no review button ('포르투갈', 'Jardim Ant?nio Borges', '폰타델가다')
----------------- 3290 -----------------
3294 url error ('포르투갈', 'Jos? do Canto Garden', '폰타델가다')
3294 no review button ('포르투갈', 'Jos? do Canto Garden', '폰타델가다')
3296 

3423 no review button ('포르투갈', 'Centro de Arte Contempor?nea Gra?a Morais', '브라간사')
3424 url error ('포르투갈', 'Pillory of Bragan?a', '브라간사')
3424 no review button ('포르투갈', 'Pillory of Bragan?a', '브라간사')
3428 url error ('포르투갈', 'Igreja de S?o Vicente', '브라간사')
3428 no review button ('포르투갈', 'Igreja de S?o Vicente', '브라간사')
3429 url error ('포르투갈', 'Bragan?a, Terra Natal e de Sonhos', '브라간사')
3429 no review button ('포르투갈', 'Bragan?a, Terra Natal e de Sonhos', '브라간사')
3430 url error ('포르투갈', 'Solar dos Cala?nhos', '브라간사')
3430 no review button ('포르투갈', 'Solar dos Cala?nhos', '브라간사')
----------------- 3430 -----------------
3432 url error ('포르투갈', 'Centro de Mem?ria Forte S. Jo?o de Deus', '브라간사')
3432 no review button ('포르투갈', 'Centro de Mem?ria Forte S. Jo?o de Deus', '브라간사')
3434 url error ('포르투갈', 'Igreja da S?', '브라간사')
3434 no review button ('포르투갈', 'Igreja da S?', '브라간사')
3435 url error ('포르투갈', 'Monte Brasil', '앙그라두에로이즈무')
3438 url error ('포르투갈', 'Cathedral of Angra do Hero?smo', '앙그라

3598 url error ('포르투갈', 'Pra?a de Touros', '나자레')
3598 no review button ('포르투갈', 'Pra?a de Touros', '나자레')
3599 url error ('포르투갈', 'Praia de Vale Furado', '나자레')
3599 no review button ('포르투갈', 'Praia de Vale Furado', '나자레')
3600 url error ('포르투갈', 'Praia da Gralha', '나자레')
----------------- 3600 -----------------
3601 url error ('포르투갈', 'Praia das Paredes', '나자레')
3602 url error ('포르투갈', 'Oficina De Turismo', '나자레')
3602 no review button ('포르투갈', 'Oficina De Turismo', '나자레')
3603 url error ('포르투갈', 'Polvoeira the beach', '나자레')
3603 no review button ('포르투갈', 'Polvoeira the beach', '나자레')
3604 url error ('포르투갈', 'Miradoro Pederneira', '나자레')
3604 no review button ('포르투갈', 'Miradoro Pederneira', '나자레')
3605 url error ('포르투갈', 'Praia da Gralha', '나자레')
3605 no review button ('포르투갈', 'Praia da Gralha', '나자레')
3608 url error ('포르투갈', 'Praia da Polvoeira', '나자레')
3610 url error ('포르투갈', 'Janela Manuelina', '나자레')
----------------- 3610 -----------------
3611 url error ('포르투갈', 'Praia das ?gu

3776 url error ('포르투갈', 'Praia de Boi?o', '포르티망')
3776 no review button ('포르투갈', 'Praia de Boi?o', '포르티망')
3778 url error ('포르투갈', 'Capela de Santa Teresa', '포르티망')
----------------- 3780 -----------------
3787 url error ('포르투갈', 'Jardim P?blico de Chaves', '샤베스')
----------------- 3790 -----------------
3791 no found ('포르투갈', 'Forte de S?o Neutel', '샤베스')
3792 url error ('포르투갈', 'Jardim do Tabolado', '샤베스')
3794 url error ('포르투갈', 'CEDIEC', '샤베스')
3794 no review button ('포르투갈', 'CEDIEC', '샤베스')
3796 url error ('포르투갈', 'Parque de Lazer do Ribeiro de Font?o', '샤베스')
3796 no review button ('포르투갈', 'Parque de Lazer do Ribeiro de Font?o', '샤베스')
3800 url error ('포르투갈', 'Solar dos Azeredo', '샤베스')
----------------- 3800 -----------------
3806 no review button ('포르투갈', 'N?cleo Museol?gico do Mar', '피게이라다포스')
3807 no review button ('포르투갈', 'N?cleo Museol?gico do Sal', '피게이라다포스')
----------------- 3810 -----------------
3814 url error ('포르투갈', 'Casa do Pa?o', '피게이라다포스')
3814 no review button (

4003 no found ('포르투갈', 'Esposende Beach', '이스포젠드')
----------------- 4010 -----------------
4011 url error ('포르투갈', 'Caixa M?gica', '이스포젠드')
4011 no review button ('포르투갈', 'Caixa M?gica', '이스포젠드')
4012 no found ('포르투갈', 'Forte de S?o Jo?o Baptista de Esposende', '이스포젠드')
4013 url error ('포르투갈', 'Praia da Bonan?a', '이스포젠드')
4013 no review button ('포르투갈', 'Praia da Bonan?a', '이스포젠드')
4014 url error ('포르투갈', 'Centro de Educa??o Ambiental', '이스포젠드')
4014 no review button ('포르투갈', 'Centro de Educa??o Ambiental', '이스포젠드')
4015 url error ('포르투갈', 'Castro de S. Mamede', '이스포젠드')
finish


In [24]:
driver.quit()

In [6]:
new_result = list(map(lambda x : (x[0], (x[1], x[2], x[3], str(x[4]))), result))

In [7]:
df_place_review = pd.DataFrame.from_items(new_result).T
print(len(df_place_review))
df_place_review.head(10)

3956


,0,1,2,3
그란비아,스페인,마드리드,"(40.420179, -3.7039276)",['']
푸에르타 데 알칼라,스페인,마드리드,"(40.419992, -3.688737)","['1778년 카를로스3세가 동쪽을 경계하기위해서 건축한, 알칼라문(개선문)입니다...."
Palacio de Cristal,스페인,마드리드,"(40.4136266, -3.6820397)",['복잡한 마드리드에서 레티로 공원은 좋은 휴식장소인 것 같습니다. 조금만 들어가도...
라스 벤타스,스페인,마드리드,"(40.4320996, -3.6633074)",['벤타스 투우장: 1929년에 만들어진 스페인 최대규모의 투우장으로 투우장 앞 광...
오리엔테 광장,스페인,마드리드,"(40.418412, -3.7121735)","['야경이 너무 멋집니다. 알무데라 대성당과 함께 꼭 한번 들러보세요 ㅎ', '리뷰..."
Parque Warner Madrid,스페인,마드리드,"(40.2302989, -3.5933781)",['(Google 번역) 다른 놀이 공원과 비교하면 Warner는 구식입니다. 롤러...
국립 고고학 박물관,스페인,마드리드,"(40.423553, -3.689402)","['""Lee Young Jun PEN ART""\n\nPen Art(General R..."
Madrid R?o,스페인,마드리드,"(40.4379543, -3.6795367)",['']
포세이돈 분수대,스페인,마드리드,"(40.4152651, -3.6941628)","['회전 교차로로 둘러 쌓여서 가까이 갈 수도 없고 멋지지도 않음', '리뷰어 ',..."
시벨레스 광장,스페인,마드리드,"(40.4195059, -3.692745)",['']


In [8]:
df_place_review.to_csv('travel_google_review_fin.csv')

In [10]:
len(err)

1863

In [19]:
err_lst = list(map(lambda x : (x[1],(x[0], x[2], x[3][:2])), err))
err_lst[0]

('Madrid R?o', ('스페인', '마드리드', ['40.4379543', '-3.6795367']))

In [20]:
err_lst[0]

('Madrid R?o', ('스페인', '마드리드', ['40.4379543', '-3.6795367']))

In [21]:
df_err = pd.DataFrame.from_items(err_lst).T
df_err.head(3)

,0,1,2
Madrid R?o,스페인,마드리드,"[40.4379543, -3.6795367]"
C?rculo de Bellas Artes,스페인,마드리드,"[40.4379543, -3.6795367]"
Isla M?gica,스페인,세비야,"[37.2928378, -5.5249689]"


In [22]:
df_err.to_csv('google_review_err.csv')

In [25]:
#에러코드 재분류

options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
#options.add_argument("disable-gpu")
options.add_argument("--disable-gpu")

driver = webdriver.Chrome('C:\\ProgramData\\Anaconda3\\Lib\\site-packages\\selenium\\webdriver\\chrome\\chromedriver.exe', chrome_options=options)
time.sleep(3)


In [27]:
df = pd.read_csv('google_review_err.csv', encoding='cp949')
df_lst = df.values.tolist()

In [29]:
df_lst[0]

['Madrid R?o', '스페인', '마드리드', "['40.4379543', '-3.6795367']"]

In [30]:
result = []
err = []
cnt = 0
print("start")

for idx in df_lst :
    place = idx[0]
    nature = idx[1]
    city = idx[2]
    err_type = 0
    
    if cnt%10 == 0 :
        print("-----------------",cnt,"-----------------")
    cnt += 1
    
    #영어 리뷰를 받을거면 ?hl=en을 붙여줘야함
    base_url = 'https://www.google.co.kr/maps/search/' + city + " " + place
    driver.get(base_url)
    time.sleep(5)
    
    #좌표받기
    latitude = 0
    longtitude = 0 
    
    url = driver.current_url
    if "/@" in url :
        try :
            url = re.search("/@(.*)z/", url).group(1).split(",")
            latitude = float(url[0])
            longtitude = float(url[1])
            
        except :
            latitude = url
            longtitude = url
            err.append((nature, place, city, url, err_type))
            print(cnt, "url error", (nature, place, city))
            
    else :
        err.append((nature, place, city, url, err_type))
        print(cnt, "no coordinate", (nature, place, city))

    #리뷰받기
    time.sleep(3)
    
    flag = False
    try :
        driver.find_element_by_css_selector('button.widget-pane-link').click()
        time.sleep(3)

    except :
        try :
            driver.find_element_by_xpath("//div[@data-result-index='1'][@class='section-result']").click()
            print(cnt, "no found", (nature, place, city))
            time.sleep(3)
            flag = True
            
        except :
            err_type=1
            print(cnt, "no review button", (nature, place, city))
            result.append([place, nature, city, (latitude, longtitude),[""]])
            err.append((nature, place, city, url, err_type))
            continue
        
    if flag : 
        try :
            driver.find_element_by_css_selector('button.widget-pane-link').click()
            time.sleep(3)
            
        except :
            err_type=2
            print(cnt, "no review button", (nature, place, city))
            result.append([place, nature, city, (latitude, longtitude),[""]])
            err.append((nature, place, city, url, err_type))
            continue
        
    
    

    for idx in range(10) :
        try :
            driver.find_element_by_css_selector('div.section-loading.noprint').click()
            time.sleep(3)
        except :
            break
    
    html =  driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    review_lst = soup.select('body > jsl > div > div > div > div > div > div > div > div > div > div > div > div > div > div > span')
    
    if len(review_lst) == 0 :
        result.append([place, nature, city, (latitude, longtitude),[""]])
    else :
        temp = []
        for review in review_lst :
            temp.append(str(review).split('>')[1].split('<')[0])
        
        result.append([place, nature, city, (latitude, longtitude), temp])
        
    if cnt%100 == 0 :
        
        new_result = list(map(lambda x : (x[0], (x[1], x[2], x[3], str(x[4]))), result))
        
        df_place_review = pd.DataFrame.from_items(new_result).T
        df_place_review.to_csv('travel_google_review' + str(cnt) + '.csv')
        print("saved")

        

print("finish")

start
----------------- 60 -----------------
61 no review button ('스페인', 'Madrid R?o', '마드리드')
62 no review button ('스페인', 'C?rculo de Bellas Artes', '마드리드')
63 no found ('스페인', 'Isla M?gica', '세비야')
63 no review button ('스페인', 'Isla M?gica', '세비야')
64 no review button ('스페인', 'Bas?lica de la Macarena', '세비야')
67 no found ('스페인', 'Plaza Bib Rambla', '그라나다')
68 no found ('스페인', 'Plaza Bib Rambla', '그라나다')
----------------- 70 -----------------
75 no found ('스페인', 'Bas?lica de Nuestra Se?ora de Las Angustias', '그라나다')
76 no found ('스페인', 'Bas?lica de Nuestra Se?ora de Las Angustias', '그라나다')
----------------- 80 -----------------
81 no found ('스페인', 'Park Federico Garc?a Lorca', '그라나다')
82 no found ('스페인', 'Park Federico Garc?a Lorca', '그라나다')
----------------- 90 -----------------
100 no found ('스페인', "Institut Valenci? d'Art Modern", '발렌시아')
saved
----------------- 100 -----------------
101 no found ('스페인', "Institut Valenci? d'Art Modern", '발렌시아')
104 no found ('스페인', 'Gonz?lez Mart? 

187 url error ('스페인', 'Esgl?sia de Sant Miquel', '팔마데마요르카')
187 no review button ('스페인', 'Esgl?sia de Sant Miquel', '팔마데마요르카')
188 url error ('스페인', 'Esgl?sia de Sant Miquel', '팔마데마요르카')
188 no review button ('스페인', 'Esgl?sia de Sant Miquel', '팔마데마요르카')
189 url error ('스페인', 'Museo Hist?rico Militar de San Carlos', '팔마데마요르카')
189 no review button ('스페인', 'Museo Hist?rico Militar de San Carlos', '팔마데마요르카')
190 url error ('스페인', 'Museo Hist?rico Militar de San Carlos', '팔마데마요르카')
190 no review button ('스페인', 'Museo Hist?rico Militar de San Carlos', '팔마데마요르카')
----------------- 190 -----------------
191 url error ('스페인', "Can' Sales", '팔마데마요르카')
191 no review button ('스페인', "Can' Sales", '팔마데마요르카')
192 url error ('스페인', "Can' Sales", '팔마데마요르카')
192 no review button ('스페인', "Can' Sales", '팔마데마요르카')
193 url error ('스페인', 'La Miseric?rdia', '팔마데마요르카')
193 no review button ('스페인', 'La Miseric?rdia', '팔마데마요르카')
194 url error ('스페인', 'La Miseric?rdia', '팔마데마요르카')
194 no review button ('스페인', 'L

258 url error ('스페인', 'Monumento al Ej?rcito de ?frica', '멜리야')
258 no review button ('스페인', 'Monumento al Ej?rcito de ?frica', '멜리야')
259 url error ('스페인', 'Casino Espa?ol', '멜리야')
259 no review button ('스페인', 'Casino Espa?ol', '멜리야')
260 url error ('스페인', 'Casino Espa?ol', '멜리야')
260 no review button ('스페인', 'Casino Espa?ol', '멜리야')
----------------- 260 -----------------
261 url error ('스페인', 'Square Four Cultures', '멜리야')
261 no review button ('스페인', 'Square Four Cultures', '멜리야')
262 url error ('스페인', 'Square Four Cultures', '멜리야')
262 no review button ('스페인', 'Square Four Cultures', '멜리야')
263 url error ('스페인', 'Fuerte de Mar?a Cristina', '멜리야')
263 no review button ('스페인', 'Fuerte de Mar?a Cristina', '멜리야')
264 url error ('스페인', 'Fuerte de Mar?a Cristina', '멜리야')
264 no review button ('스페인', 'Fuerte de Mar?a Cristina', '멜리야')
265 url error ('스페인', 'Compa?ia Melillense de Gas y Electricidad, S.A', '멜리야')
265 no review button ('스페인', 'Compa?ia Melillense de Gas y Electricidad, S.A

373 url error ('스페인', 'Isabel la Cat?lica Park', '히혼')
373 no review button ('스페인', 'Isabel la Cat?lica Park', '히혼')
374 url error ('스페인', 'Isabel la Cat?lica Park', '히혼')
374 no review button ('스페인', 'Isabel la Cat?lica Park', '히혼')
375 url error ('스페인', 'Iglesia de San Pedro Ap?stol', '히혼')
375 no review button ('스페인', 'Iglesia de San Pedro Ap?stol', '히혼')
376 url error ('스페인', 'Iglesia de San Pedro Ap?stol', '히혼')
376 no review button ('스페인', 'Iglesia de San Pedro Ap?stol', '히혼')
377 url error ('스페인', 'Playa de Espa?a', '히혼')
377 no review button ('스페인', 'Playa de Espa?a', '히혼')
378 url error ('스페인', 'Playa de Cand?s', '히혼')
378 no review button ('스페인', 'Playa de Cand?s', '히혼')
379 url error ('스페인', 'Playa de Pe?arrubia', '히혼')
379 no review button ('스페인', 'Playa de Pe?arrubia', '히혼')
----------------- 380 -----------------
382 url error ('스페인', 'Rincon?n', '히혼')
382 no review button ('스페인', 'Rincon?n', '히혼')
383 url error ('스페인', 'Rincon?n', '히혼')
383 no review button ('스페인', 'Rinc

447 url error ('스페인', 'O Farol de Urz?iz', '비고')
447 no review button ('스페인', 'O Farol de Urz?iz', '비고')
448 url error ('스페인', 'O Farol de Urz?iz', '비고')
448 no review button ('스페인', 'O Farol de Urz?iz', '비고')
449 url error ('스페인', 'Xard?ns hist?ricos do Pazo de Castrelos', '비고')
449 no review button ('스페인', 'Xard?ns hist?ricos do Pazo de Castrelos', '비고')
450 url error ('스페인', 'Xard?ns hist?ricos do Pazo de Castrelos', '비고')
450 no review button ('스페인', 'Xard?ns hist?ricos do Pazo de Castrelos', '비고')
----------------- 450 -----------------
451 url error ('스페인', 'Porta do Atl?ntico', '비고')
451 no review button ('스페인', 'Porta do Atl?ntico', '비고')
452 url error ('스페인', 'Porta do Atl?ntico', '비고')
452 no review button ('스페인', 'Porta do Atl?ntico', '비고')
453 url error ('스페인', 'Centro de Recepci?n e Interpretaci?n del Prerrom?nico Asturiano', '오비에도')
453 no review button ('스페인', 'Centro de Recepci?n e Interpretaci?n del Prerrom?nico Asturiano', '오비에도')
454 url error ('스페인', 'Centro de Rece

...

1634 url error ('포르투갈', 'Jos? Malhoa Museum', '오비두스')
1634 no review button ('포르투갈', 'Jos? Malhoa Museum', '오비두스')
1635 url error ('포르투갈', 'Museu de Cer?mica', '오비두스')
1635 no review button ('포르투갈', 'Museu de Cer?mica', '오비두스')
1636 url error ('포르투갈', 'Museu de Cer?mica', '오비두스')
1636 no review button ('포르투갈', 'Museu de Cer?mica', '오비두스')
1637 url error ('포르투갈', 'Igreja de Nossa Senhora do P?pulo', '오비두스')
1637 no review button ('포르투갈', 'Igreja de Nossa Senhora do P?pulo', '오비두스')
1638 url error ('포르투갈', 'Parque Cineg?tico de ?bidos', '오비두스')
1638 no review button ('포르투갈', 'Parque Cineg?tico de ?bidos', '오비두스')
1639 url error ('포르투갈', 'Parque Cineg?tico de ?bidos', '오비두스')
1639 no review button ('포르투갈', 'Parque Cineg?tico de ?bidos', '오비두스')
1640 url error ('포르투갈', 'Pal?cio Real', '오비두스')
1640 no review button ('포르투갈', 'Pal?cio Real', '오비두스')
----------------- 1640 -----------------
1641 url error ('포르투갈', 'Pal?cio Real', '오비두스')
1641 no review button ('포르투갈', 'Pal?cio Real', '오비두스')
1

1701 url error ('포르투갈', 'Praia das ?guas Luxuosas', '나자레')
1701 no review button ('포르투갈', 'Praia das ?guas Luxuosas', '나자레')
1702 url error ('포르투갈', 'Museu Monogr?fico do B?rrio', '나자레')
1702 no review button ('포르투갈', 'Museu Monogr?fico do B?rrio', '나자레')
1703 url error ('포르투갈', 'Museu Monogr?fico do B?rrio', '나자레')
1703 no review button ('포르투갈', 'Museu Monogr?fico do B?rrio', '나자레')
1704 url error ('포르투갈', 'Alojamento Local Quinta Do Val?o', '나자레')
1704 no review button ('포르투갈', 'Alojamento Local Quinta Do Val?o', '나자레')
1705 url error ('포르투갈', 'Alojamento Local Quinta Do Val?o', '나자레')
1705 no review button ('포르투갈', 'Alojamento Local Quinta Do Val?o', '나자레')
1706 url error ('포르투갈', 'Praia de Vale Pardo', '나자레')
1706 no review button ('포르투갈', 'Praia de Vale Pardo', '나자레')
1707 url error ('포르투갈', 'Praia de Vale Pardo', '나자레')
1707 no review button ('포르투갈', 'Praia de Vale Pardo', '나자레')
1708 url error ('포르투갈', 'Gr?o Vasco Museum', '비제우')
1708 no review button ('포르투갈', 'Gr?o Vasco Museum

1770 url error ('포르투갈', 'Aqu?rio do Porto Pim - Esta??o de Peixes Vivos', '오르타')
1770 no review button ('포르투갈', 'Aqu?rio do Porto Pim - Esta??o de Peixes Vivos', '오르타')
----------------- 1770 -----------------
1771 url error ('포르투갈', 'Aqu?rio do Porto Pim - Esta??o de Peixes Vivos', '오르타')
1771 no review button ('포르투갈', 'Aqu?rio do Porto Pim - Esta??o de Peixes Vivos', '오르타')
1772 no review button ('포르투갈', 'F?brica da Baleia', '오르타')
1773 url error ('포르투갈', 'Torre do Rel?gio', '오르타')
1773 no review button ('포르투갈', 'Torre do Rel?gio', '오르타')
1774 url error ('포르투갈', 'Torre do Rel?gio', '오르타')
1774 no review button ('포르투갈', 'Torre do Rel?gio', '오르타')
1775 url error ('포르투갈', 'Guarita', '오르타')
1775 no review button ('포르투갈', 'Guarita', '오르타')
1776 url error ('포르투갈', 'Casa Mem?ria Manuel de Arriaga', '오르타')
1776 no review button ('포르투갈', 'Casa Mem?ria Manuel de Arriaga', '오르타')
1777 url error ('포르투갈', 'Tr?s Irm?os', '포르티망')
1777 no review button ('포르투갈', 'Tr?s Irm?os', '포르티망')
1778 url error 

1836 url error ('포르투갈', 'Pillory of Alhos Vedros', '몬티주')
1836 no review button ('포르투갈', 'Pillory of Alhos Vedros', '몬티주')
1837 url error ('포르투갈', 'Pillory of Alhos Vedros', '몬티주')
1837 no review button ('포르투갈', 'Pillory of Alhos Vedros', '몬티주')
1838 url error ('포르투갈', 'Pal?cio do Vidigal', '몬티주')
1838 no review button ('포르투갈', 'Pal?cio do Vidigal', '몬티주')
1839 url error ('포르투갈', 'Coreto de Santo Est?v?o', '몬티주')
1839 no review button ('포르투갈', 'Coreto de Santo Est?v?o', '몬티주')
1840 url error ('포르투갈', 'Coreto de Santo Est?v?o', '몬티주')
1840 no review button ('포르투갈', 'Coreto de Santo Est?v?o', '몬티주')
----------------- 1840 -----------------
1841 url error ('포르투갈', 'Filipe Palho?a Vinhos', '몬티주')
1841 no review button ('포르투갈', 'Filipe Palho?a Vinhos', '몬티주')
1842 url error ('포르투갈', 'Filipe Palho?a Vinhos', '몬티주')
1842 no review button ('포르투갈', 'Filipe Palho?a Vinhos', '몬티주')
1843 url error ('포르투갈', 'Casa das Hist?rias Paula Rego', '이스토릴')
1843 no review button ('포르투갈', 'Casa das Hist?rias 

1901 url error ('포르투갈', 'Villa Campus', '산타렝')
1901 no review button ('포르투갈', 'Villa Campus', '산타렝')
1902 url error ('포르투갈', 'Casa Agr?cola Quinta Do Falc?o Lda.', '산타렝')
1902 no review button ('포르투갈', 'Casa Agr?cola Quinta Do Falc?o Lda.', '산타렝')
1903 url error ('포르투갈', 'Casa Agr?cola Quinta Do Falc?o Lda.', '산타렝')
1903 no review button ('포르투갈', 'Casa Agr?cola Quinta Do Falc?o Lda.', '산타렝')
1904 url error ('포르투갈', 'Fluvi?rio de Mora', '아비스')
1904 no review button ('포르투갈', 'Fluvi?rio de Mora', '아비스')
1905 url error ('포르투갈', 'Fluvi?rio de Mora', '아비스')
1905 no review button ('포르투갈', 'Fluvi?rio de Mora', '아비스')
1906 url error ('포르투갈', 'anta de Pavia', '아비스')
1906 no review button ('포르투갈', 'anta de Pavia', '아비스')
1907 url error ('포르투갈', 'anta de Pavia', '아비스')
1907 no review button ('포르투갈', 'anta de Pavia', '아비스')
1908 url error ('포르투갈', 'Casa Museu Manuel Ribeiro de Pavia', '아비스')
1908 no review button ('포르투갈', 'Casa Museu Manuel Ribeiro de Pavia', '아비스')
1909 url error ('포르투갈', 'Casa Mu

# 위키피디아(영문)

### 영문 관광지 가져오기

In [2]:
#셀리옴으로 크롤링하기

options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
#options.add_argument("disable-gpu")
options.add_argument("--disable-gpu")

In [3]:
driver = webdriver.Chrome('C:\\ProgramData\\Anaconda3\\Lib\\site-packages\\selenium\\webdriver\\chrome\\chromedriver.exe', chrome_options=options)
time.sleep(3)

In [4]:
#나라별 도시 크롤링
country =  ["스페인", "포르투갈"]
dic = {}
for idx in country :
    dic[idx] = []

In [5]:
tot = 0
for nature in country :
    base_url = 'https://www.google.co.kr/search?' +'q=' + nature + '+도시&hl=en'
    driver.get(base_url)
    time.sleep(3)
    
    lst = driver.find_elements_by_css_selector("a.klitem")
    for city in lst :
        dic[nature].append(city.text.split("\n")[0])
        tot += 1
    
print(tot)

102


In [ ]:
result = []
err_lst = []

regex1 = ">(.*)<"
regex2 = "Rated (.*) out"
regex3 = "\((.*)\)"
regex4 = ">(.*)<"
regex5 = ""

tot = 1
a = 0
total = 0
for nature in country :
    for city in dic[nature] :
        print(tot, nature, city)
        a+=1
        
        try : 
            #hl=en 을 넣어주면 영어 검색
            base_url = 'https://www.google.co.kr/search?' +'q=' + nature + '+' + city + '+' + "관광지&hl=en"
            driver.get(base_url)
            time.sleep(5)
            
            #mor_buttin_id가 수시로 바뀌니 체크하기
            more_button_id = '_x7n'
            driver.find_element_by_class_name(more_button_id).click()
            time.sleep(5)

            html =  driver.page_source
            soup = BeautifulSoup(html, "html.parser")
            temp = soup.select("body > div > div > div > ol > li > div > div")
            
            total = 0
            for idx, place in enumerate(temp) :
                soup = BeautifulSoup(str(place), "html.parser")
                
                name = re.search(regex1, str(soup.select("div > h2")[0])).group(1)
                
                try : star = float(re.search(regex2, str(soup.select("div >  g-review-stars > span")[0])).group(1))
                except : star = 0
                    
                try : review = int(re.search(regex3, str(soup.select("div > div  > span._Mnc")[0])).group(1).replace(",", ""))
                except : review = 0
                
                try : short_info = re.search(regex4, str(soup.select("p._DAm")[0])).group(1)
                except : short_info = ""
                    
                try : long_info = str(soup.select("p._BAm")[0]).split(">")[1].split("<")[0]
                except : long_info = ""
                
                #위키피디아 검색
                
                driver.find_elements_by_class_name("_Qzm")[idx].click()
                time.sleep(10)
                
                try :
                    driver.find_element_by_class_name("q").click()
                    time.sleep(10)
                except :
                    time.sleep(10)
                    
                    try :
                        driver.find_element_by_class_name("q").click()
                        time.sleep(5)
                    except :
                        temp_html = driver.page_source
                        temp_soup = BeautifulSoup(temp_html, "html.parser")
                        contents = temp_soup.select("body > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div > span")
                    
                        result.append((name, (nature, city, star, review, short_info, long_info, contents)))
                        total += 1
                    
                        print("   ", name,"err_no_wiki")
                        err_lst.append((nature, city))
                        continue
                
                wiki_html = driver.page_source
                soup2 = BeautifulSoup(wiki_html, "html.parser")
                try :
                    contents = soup2.select("body > div > div > div > div > p")
                except :
                    print("   ","err_wiki")
                    err_lst.append((nature, city))
                    contents = []
    
                driver.back()
                time.sleep(3)
            
                result.append((name, (nature, city, star, review, short_info, long_info, contents)))
                total += 1
                
            print("***",total, "clear")
        except :
            err_lst.append((nature, city))
            print("   ","err_fin")
        

        tot+=1
    
    

        

1 스페인 Madrid
    Gran Vía err_no_wiki
    Plaza de Cibeles err_no_wiki
    Madrid Río err_no_wiki
    plaza de Cánovas del Castillo err_no_wiki
    Barrio de La Latina err_no_wiki
    Cybele Palace err_no_wiki
    Plaza de Santa Ana err_no_wiki
    Lavapiés err_no_wiki
    Calle de Alcalá err_no_wiki
    Bernabéu Tour err_no_wiki
*** 50 clear
2 스페인 Barcelona
    La Rambla, Barcelona err_no_wiki
    Gothic Quarter, Barcelona err_no_wiki
    Montjuïc err_no_wiki
    Montserrat err_no_wiki
    La Barceloneta err_no_wiki
    Tibidabo err_no_wiki
    Plaça Reial err_no_wiki
    Port of Barcelona err_no_wiki
    Avinguda Diagonal err_no_wiki
    Museum of the History of Catalonia err_no_wiki
    Passeig de Gràcia, Barcelona err_no_wiki
*** 50 clear
3 스페인 Seville
    Flamenco Dance Museum err_no_wiki
    Palace of San Telmo err_no_wiki
    La Alameda, Seville err_no_wiki
    plaza Nueva err_no_wiki
    Sevilla Free Tours err_no_wiki
    Antiquarium err_no_wiki
    Torre Giralda err_no_wiki
  

    Bullfighting Museum of Cordoba err_no_wiki
    Zoo Cordoba err_no_wiki
    Centro Flamenco Fosforito err_no_wiki
    Casa Ramón García Romero (antes Casa Museo Arte sobre Piel) err_no_wiki
    Museo Vivo de Al-Andalus err_no_wiki
    Monument to the Lovers (Iben Zaydon and Wallada) err_no_wiki
    Palacio de Viana err_no_wiki
    Plaza de la Corredera err_no_wiki
    Plaza de las Tendillas err_no_wiki
    Royal Botanical Garden Cordoba err_no_wiki
    Molino de la Albolafia err_no_wiki
    Casa de las Cabezas err_no_wiki
    Los Patios del Alcazar Viejo err_no_wiki
    Posada Del Potro err_no_wiki
    Sotos de la Albolafia err_no_wiki
    Patio De Los Naranjos err_no_wiki
    Triunfo de San Rafael de la Puerta del Puente err_no_wiki
    Murallas y Puertas de Almodovar err_no_wiki
    Casa de las Cabezas Museo-Patios err_no_wiki
    Casa Árabe err_no_wiki
    Baños De Popea err_no_wiki
    Las Ermitas Cordoba err_no_wiki
    Jardines de la Victoria err_no_wiki
    Jardines Del Alcaz

    La Clerecía err_no_wiki
    Jesuitas Park err_no_wiki
    Anaya Palace err_no_wiki
    Cueva de Salamanca err_no_wiki
    Palacio de la Salina err_no_wiki
    Iglesia de San Martín err_no_wiki
    Frog Salamanca err_no_wiki
    Iglesia de San Marcos err_no_wiki
    House of Abarca err_no_wiki
    Pozo de las Nieves err_no_wiki
    Cerro de San Vicente err_no_wiki
    Casino De Salamanca err_no_wiki
    Casa de las Muertes err_no_wiki
    Monumenta Salamanticae (Iglesia de San Millán) err_no_wiki
    Plaza de Anaya err_no_wiki
    Iglesia de San Juan de Sahagún err_no_wiki
    Iglesia de la Purísima err_no_wiki
    Verraco del puente err_no_wiki
    Monumento al Lazarillo de Tormes err_no_wiki
    Iglesia de San Juan de Barbalos err_no_wiki
    Casa de Santa Teresa de Jesús err_no_wiki
    Würzburg Park err_no_wiki
    Torre del Marqués de Villena err_no_wiki
    Dona María la Brava err_no_wiki
    Casa de las Viejas err_no_wiki
    Incoming Salamanca err_no_wiki
    Asociación de C

In [ ]:
driver.quit()

In [ ]:
df_city_google_info = pd.DataFrame.from_items(result).T

In [ ]:
print(len(df_city_google_info))
df_city_google_info.head()

In [ ]:
df_city_google_info.tail(10)

In [ ]:
print(len(err_lst))
err_lst

In [ ]:
df_city_google_info.to_csv('travel_google_en2.csv')

### 위키피디아

In [ ]:
lst = list(map(lambda x : (x[0], x[1][0], x[1][1]), result))
lst[0]

In [ ]:
for idx, place in enumerate(lst) :
    name = place[0]
    try :
        page = wk.page(name)
        summary = page.summary
        content = page.content
    except :
        content = ""
    
    lst[idx] = (place[0],(place[1], place[2], content, summary))

In [ ]:
df_city_wiki_info = pd.DataFrame.from_items(lst).T

In [ ]:
print(len(df_city_wiki_info))
df_city_wiki_info.head()

In [ ]:
df_city_wiki_info.tail()

In [ ]:
df_city_wiki_info.to_csv('travel_wiki_en1.csv')

# 결과 확인

In [ ]:
df =  pd.read_csv("travel_google_en2.csv")

In [ ]:
df = df.rename(columns = {"Unnamed: 0" : "name", "0": "nature","1" : "city", "2" : "star", "3": "reviews", "4" : "short_info", "5" : "long_info", "6" : "wiki"})
df = df.set_index("name")
df.head()

In [ ]:
df2 = pd.read_csv("travel_wiki_en1.csv")

In [ ]:
df2 = df2.rename(columns = {"Unnamed: 0" : "name", "0": "nature","1" : "city", "2" : "wiki", "3" : "wiki_summary"})
df2 = df2.set_index("name")
df2.head()

# 연습코드

requests_url = "http://www.foodsafetykorea.go.kr/portal/healthyfoodlife/selectCalorieDicList.do"

payload = dict()
payload['menu_no'] = '3072'
payload['menu_grp'] = 'MENU_NEW03'
payload['copyUrl'] = 'http://www.foodsafetykorea.go.kr:80/portal/healthyfoodlife/calorieDic.do?menu_no=3072&menu_grp=MENU_NEW03'
payload['grp_option'] = 'FOOD GROUP CODE'
payload['code2'] = ''
payload['nutrition-keyword'] = '한글/영어/이명'
payload['search_name'] = ''
payload['start_idx'] = 1
payload['show_cnt'] = 20
payload['order_column'] = 'FOOD_CD'
payload['order_sending'] = 'ASC'

r = requests.post(requests_url, data=payload)

print(r.status_code)
print(r.json())

driver = webdriver.Chrome('C:\\ProgramData\\Anaconda3\\Lib\\site-packages\\selenium\\webdriver\\chrome\\chromedriver.exe')
driver.implicitly_wait(3)

base_url = 'https://www.google.co.kr/maps/search/바르셀로나+관광지/@40.4085776,-3.7263944,13z/data=!4m3!2m2!5m1!4e1'
driver.get(base_url)
driver.implicitly_wait(3)

search_input_id = 'searchboxinput'
place_name = '마드리드'
driver.find_element_by_id(search_input_id).send_keys(place_name + " 관광지")

search_button_id = 'searchbox-searchbutton'
driver.find_element_by_id(search_button_id).click()

driver.implicitly_wait(3)

detail_button_css = 'button.section-filters-filter'
driver.find_element_by_css_selector(detail_button_css).click()

#NoVisibleError
star_button_id =  'section-filters-toggle-option-id-1'
driver.find_element_by_id(star_button_id).click()

temp = 'div.section-filters-toggle-option'
driver.find_element_by_css_selector(temp).click()

temp = WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.ID, "section-filters-toggle-option-id-1")))
temp.clear()
temp.click()

finish_button_css = 'button.section-filters-header-save-button.ripple-container blue-button-text'
driver.find_element_by_css_selector(finish_button_css).click()

url = driver.current_url
req = requests.get(url + '/data=!4m3!2m2!5m1!4e1')
html = req.text
soup = BeautifulSoup(html, 'html.parser')
place_lst = soup.select("body > jsl > div#app-container > div#content-container > div#pane > div.widget-pane.widget-pane-visible > div.widget-pane-content.scrollable-y > div.widget-pane-content-holder > div.section-listbox.section-listbox-root > div > div.section-result > div > div.section-result-header > div > h3")
review_lst = soup.select("body > jsl > div#app-container > div#content-container > div#pane > div.widget-pane.widget-pane-visible > div.widget-pane-content.scrollable-y > div.widget-pane-content-holder > div.section-listbox.section-listbox-root > div > div.section-result > div > div.section-result-header > div > span > span.section-result-num-ratings")
star_lst = soup.select("body > jsl > div#app-container > div#content-container > div#pane > div.widget-pane.widget-pane-visible > div.widget-pane-content.scrollable-y > div.widget-pane-content-holder > div.section-listbox.section-listbox-root > div > div.section-result > div > div.section-result-header > div > span > span > span")



driver = webdriver.Chrome('C:\\ProgramData\\Anaconda3\\Lib\\site-packages\\selenium\\webdriver\\chrome\\chromedriver.exe')
driver.implicitly_wait(3)

base_url = 'https://www.google.co.kr/search?' +'q=스페인+도시&hl=en'
driver.get(base_url)
driver.implicitly_wait(3)

html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
place_lst = soup.select("body > div > div > div > div > div > div > div > div > div > div > g-scrolling-carousel > div > div > div > a > div > div.kltat > span")

driver = webdriver.Chrome('C:\\ProgramData\\Anaconda3\\Lib\\site-packages\\selenium\\webdriver\\chrome\\chromedriver.exe')
driver.implicitly_wait(3)

base_url = 'https://www.google.co.kr/maps/search/' + 'Barcelona' + '+관광지/data=!4m3!2m2!5m1!4e1'
driver.get(base_url)
driver.implicitly_wait(10)

html =  driver.page_source
soup = BeautifulSoup(html, "html.parser")

place_lst = soup.select("body > jsl > div#app-container > div#content-container > div#pane > div.widget-pane.widget-pane-visible > div.widget-pane-content.scrollable-y > div.widget-pane-content-holder > div.section-listbox.section-listbox-root > div > div.section-result > div > div.section-result-header > div > h3")
review_lst = soup.select("body > jsl > div#app-container > div#content-container > div#pane > div.widget-pane.widget-pane-visible > div.widget-pane-content.scrollable-y > div.widget-pane-content-holder > div.section-listbox.section-listbox-root > div > div.section-result > div > div.section-result-header > div > span > span.section-result-num-ratings")
star_lst = soup.select("body > jsl > div#app-container > div#content-container > div#pane > div.widget-pane.widget-pane-visible > div.widget-pane-content.scrollable-y > div.widget-pane-content-holder > div.section-listbox.section-listbox-root > div > div.section-result > div > div.section-result-header > div > span > span > span")

print(place_lst)

next_button_id = 'section-pagination-button-next'
inputElement = driver.find_element_by_id(next_button_id)
inputElement.send_keys("\n")
time.sleep(5)

html =  driver.page_source
soup = BeautifulSoup(html, "html.parser")

place_lst = soup.select("body > jsl > div#app-container > div#content-container > div#pane > div.widget-pane.widget-pane-visible > div.widget-pane-content.scrollable-y > div.widget-pane-content-holder > div.section-listbox.section-listbox-root > div > div.section-result > div > div.section-result-header > div > h3")
review_lst = soup.select("body > jsl > div#app-container > div#content-container > div#pane > div.widget-pane.widget-pane-visible > div.widget-pane-content.scrollable-y > div.widget-pane-content-holder > div.section-listbox.section-listbox-root > div > div.section-result > div > div.section-result-header > div > span > span.section-result-num-ratings")
star_lst = soup.select("body > jsl > div#app-container > div#content-container > div#pane > div.widget-pane.widget-pane-visible > div.widget-pane-content.scrollable-y > div.widget-pane-content-holder > div.section-listbox.section-listbox-root > div > div.section-result > div > div.section-result-header > div > span > span > span")

print(place_lst)

next_button_id = 'section-pagination-button-next'
inputElement = driver.find_element_by_id(next_button_id)
inputElement.send_keys("\n")
time.sleep(5)

html =  driver.page_source
soup = BeautifulSoup(html, "html.parser")

place_lst = soup.select("body > jsl > div#app-container > div#content-container > div#pane > div.widget-pane.widget-pane-visible > div.widget-pane-content.scrollable-y > div.widget-pane-content-holder > div.section-listbox.section-listbox-root > div > div.section-result > div > div.section-result-header > div > h3")
review_lst = soup.select("body > jsl > div#app-container > div#content-container > div#pane > div.widget-pane.widget-pane-visible > div.widget-pane-content.scrollable-y > div.widget-pane-content-holder > div.section-listbox.section-listbox-root > div > div.section-result > div > div.section-result-header > div > span > span.section-result-num-ratings")
star_lst = soup.select("body > jsl > div#app-container > div#content-container > div#pane > div.widget-pane.widget-pane-visible > div.widget-pane-content.scrollable-y > div.widget-pane-content-holder > div.section-listbox.section-listbox-root > div > div.section-result > div > div.section-result-header > div > span > span > span")

print(place_lst)